In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [ ]:
df_market_data.columns

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaler = StandardScaler()
scaler.fit(df_market_data)
results = scaler.transform(df_market_data)


In [ ]:
# Create a DataFrame with the scaled data
scaled_data_df = pd.DataFrame(results, columns=['price_change_percentage_24h', 'price_change_percentage_7d',
       'price_change_percentage_14d', 'price_change_percentage_30d',
       'price_change_percentage_60d', 'price_change_percentage_200d',
       'price_change_percentage_1y'])

# Copy the crypto names from the original data
scaled_data_df['coin_id'] = df_market_data.index.values

# Set the coinid column as index
scaled_data_df = scaled_data_df.set_index('coin_id', drop=True)

# Display sample data
scaled_data_df.head(10)

---

### Find the Best Value for k Using the Original Data.

In [ ]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,12))

In [ ]:
# Create an empty list to store the inertia values
inertia=[]

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    model = KMeans(n_clusters=i, random_state=1)
    model.fit(scaled_data_df)
    inertia.append(model.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
data = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(data)
elbow_df.head()

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_df.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve - Crypto Clustering",
    xticks=k
)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 4 ... the slope of the graph moves most towards flat at a cluster count of 4.

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [ ]:
# Initialize the K-Means model using the best value for k
k_model = KMeans(n_clusters=4, random_state=1)

In [ ]:
# Fit the K-Means model using the scaled data
k_model.fit(scaled_data_df)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
k_clusters = k_model.predict(scaled_data_df)

# Print the resulting array of cluster values.
k_clusters

In [ ]:
# Create a copy of the DataFrame
scaled_copy_df = scaled_data_df.copy()

In [ ]:
# Add a new column to the DataFrame with the predicted clusters
scaled_copy_df['k_clusters'] = k_clusters

# Display sample data
scaled_copy_df.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
scaled_copy_df.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by='k_clusters',
    hover_cols="coin_id"
)

---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
principal=PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
components = principal.fit_transform(scaled_data_df)

# View the first five rows of the DataFrame. 
components[:5]

In [ ]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
principal.explained_variance_ratio_

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** About 87% of the total variance is codensed into the 3 PCA variables.

In [ ]:
# Create a new DataFrame with the PCA data.
pca_df = pd.DataFrame(components, columns=['PCA1', 'PCA2', 'PCA3'])

# Copy the crypto names from the original data
pca_df['coin_id'] = scaled_copy_df.index.values

# Set the coinid column as index
pca_df = pca_df.set_index('coin_id', drop=True)

# Display sample data
pca_df.head()

---

### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values from 1 to 11
k_pca = list(range(1, 12))

In [ ]:
# Create an empty list to store the inertia values
inertia_pca = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k_pca:
    model = KMeans(n_clusters=i, random_state=1)
    model.fit(pca_df)
    inertia_pca.append(model.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
data_pca = {"k": k_pca, "inertia": inertia_pca}

# Create a DataFrame with the data to plot the Elbow curve
elbow_pca_df = pd.DataFrame(data_pca)

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_pca_df.hvplot.line(
    x="k",
    y="inertia",
    title="PCA Elbow Graph",
    xticks=k_pca
)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No ... significant slope change occurs at the same place.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialize the K-Means model using the best value for k
best_model = KMeans(n_clusters=4, random_state=1)

In [ ]:
# Fit the K-Means model using the PCA data
best_model.fit(pca_df)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data
pca_clusters = best_model.predict(pca_df)

# Print the resulting array of cluster values.
pca_clusters

In [ ]:
# Create a copy of the DataFrame with the PCA data
best_pca_df = pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters
best_pca_df['best_clusters'] = pca_clusters

# Display sample data
best_pca_df.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
best_pca_df.hvplot.scatter(
    x="PCA1",
    y="PCA2",
    by='best_clusters',
    hover_cols="coin_id"
)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
# Composite plot to contrast the Elbow curves
elbow_df.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve - Crypto Clustering",
    xticks=k,
    line_color='red'
) * elbow_pca_df.hvplot.line(
    x="k",
    y="inertia",
    title="PCA Elbow Graph",
    xticks=k_pca,
    line_color='blue'
)

In [ ]:
# Composite plot to contrast the clusters
scaled_copy_df.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by='k_clusters',
    hover_cols="coin_id",
    title="Full Feature Clustering"
) + best_pca_df.hvplot.scatter(
    x="PCA1",
    y="PCA2",
    by='best_clusters',
    hover_cols="coin_id",
    title="PCA Data Clustering"
)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** The PCA data clearly shows the outliers (clusters 2 & 3) as outliers, while providing similar clarity for clusters 0 & 1. Without the reduction in features, there is more overlap (see full data, clusters 1 & 2).